<a href="https://colab.research.google.com/github/oscarWoj/CISC499/blob/main/csv_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parsing Conversations from a scraped forum
The steps are:
- Data Import (from google drive currently)
- Inital Parse, select relevent data and save
- Read Data back from CSV and parse thoroughly
  - save results

In [1]:
import csv
import pandas as pd
from nltk.stem import SnowballStemmer
!pip install gdown
!pip install --upgrade gdown
import gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


## Data Import

This replaces the need to import Forum157.7z into the machine

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/164ZTc7EB4my1mRBVv-K-tREx4EMDmKY8/view?usp=share_link

Downloading...
From: https://drive.google.com/uc?id=164ZTc7EB4my1mRBVv-K-tREx4EMDmKY8
To: /content/Forum157.7z
100% 1.14G/1.14G [00:13<00:00, 88.0MB/s]


In [3]:
!mkdir /content/parsedData
!mkdir /content/data

#Untar into data folder
!7za x -o/content/data -y Forum157.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1144004002 bytes (1092 MiB)

Extracting archive: Forum157.7z
--
Path = Forum157.7z
Type = 7z
Physical Size = 1144004002
Headers Size = 398
Method = LZMA2:26
Solid = +
Blocks = 5

  0%      0% - BlackHatWorld.txt_Part0.txt                                    1% - BlackHatWorld.txt_Part0.txt                                    2% - BlackHatWorld.txt_Part0.txt                                    3% - BlackHatWorld.txt_Part0.txt                                  

## Inital Parse, select relevent data (text body)

In [4]:
#Converts each (of the 11) file from .txt to .csv
for i in range(11):
  path = "/content/data/BlackHatWorld.txt_Part"+str(i)+".txt"
  read_file = pd.read_csv (path,sep='\t')
  # This file is seperated by /t spaces

  #Select only the text
  read_file = read_file.filter(items=["txtBody_Clean"])

  #Save into a new 'parsed' csv in parsedData directory
  savePath = "/content/parsedData/parsedText_part"+str(i)+".csv"
  read_file.to_csv (savePath, index=None)
  print(i)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,8,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1
2
3
4
5
6
7
8


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


9
10


In [5]:
del read_file # Frees up some memory ( i hope )

In [6]:
# Concatonate all the csv's together
!cat /content/parsedData/parsedText_part*.csv > outputfile.csv

##Read Data back from CSV and parse thoroughly

Read the data back into a dataframe, then use pandas dataframe to do our parsing techniques

In [7]:
df = pd.read_csv('/content/outputfile.csv', header=None, names=['text'])

In [8]:
print(df)

                                                       text
0                                             txtBody_Clean
1         Cloaking is a search engine optimization techn...
2         Do the search engines allow this. If they disc...
3         Often when you get a high ranking page under q...
4         There are 5 types of cloaking:        * User A...
...                                                     ...
10015344  I saw this about a year ago. Without looking t...
10015345  Yes Pxoxrxn, i totally agree that we're advanc...
10015346  pxoxrxn said:                      ↑          ...
10015347  Just for anyone that is interested, this guy h...
10015348  It looks mostly geared towards people that hav...

[10015349 rows x 1 columns]


In [9]:
# Remove the rows containing NaN values
df = df.dropna()

In [10]:
print(len(df))

9976012


In [11]:
total_chars = df.applymap(lambda x: len(str(x))).sum().sum()
print(total_chars)

3598866267


In [12]:
# Remove the rows that contain the string header "txtBody_Clean"
df = df[~df['text'].str.contains('txtBody_Clean')]

In [13]:
# Remove the rows containing NaN values
df = df.dropna()

In [14]:
print(len(df))

9976001


In [41]:
#This will not truncate long text, to better visualize the parse
# Access a row of the DataFrame
row = df.iloc[44473]

# Set the maximum column width to display
pd.options.display.max_colwidth = None

print(row)

text    if only their was someone that sold an archive of premium wordpress themes on this site  that advertised on this site  unfortunately their is no one on this site that does 
Name: 44515, dtype: object


In [26]:
df['text'] = df['text'].str.split("var dark_postrating", n=1, expand=True)[0]
#Cut out rest of the string afer footer text

In [27]:
# Pattern matching replies from previous posts
pattern = 'said:.*?Click to expand\.'

# Use str.replace to remove the matching pattern
df['text'] = df['text'].str.replace(pattern, '', regex=True)

In [28]:
#replace links with simpler version i.e. -> https://www.pinecone.io/learn/bertopic/ -> httppineconeio/learn/bertopic
# Replace the URLs with simplified form
df['text'] = df['text'].str.replace('https://www.', 'http')
df['text'] = df['text'].str.replace('/', '')

<ipython-input-28-2341122a913b>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('https://www.', 'http')


In [29]:
# replace non-alphanumeric characters with spaces
df['text'] = df['text'].str.replace('[^\w\s]', ' ')

<ipython-input-29-970d14caf20f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', ' ')


In [30]:
# Use str.lower to convert each string to lowercase
df['text'] = df['text'].str.lower()

In [31]:
df.to_csv (r'/content/output_simple.csv', index=None)

In [32]:
!zip output_simple.csv.zip output_simple.csv

  adding: output_simple.csv (deflated 71%)


In [33]:
total_chars2 = df.applymap(lambda x: len(str(x))).sum().sum()
print(total_chars2)

2699001424


In [34]:
#Stemming is a technique used in natural language processing to reduce inflectional 
#and derivational forms of words to their base or root form.
#Stemming is the process of removing the suffixes from words to reduce them to their stem.
# Initialize Snowball Stemmer
stemmer = SnowballStemmer("english")

# Stem every row in the DataFrame
df['text'] = df['text'].apply(lambda x: stemmer.stem(x))

In [35]:
df.to_csv (r'/content/output_complex.csv', index=None)

In [36]:
!zip -dd output_complex.csv.zip output_complex.csv

  adding: output_complex.csv ................................................................................................................................................................................................................................................................... (deflated 71%)


In [37]:
total_chars3 = df.applymap(lambda x: len(str(x))).sum().sum()
print(total_chars3)

2696812496
